In [0]:
%run /Shared/storage_access


[FileInfo(path='abfss://bronze@armazenamentodatalake26.dfs.core.windows.net/SalesLT/', name='SalesLT/', size=0, modificationTime=1768328136000)]

In [0]:
#salvar o nome das tabelas em um lista
table_name = []

for i in dbutils.fs.ls("abfss://bronze@armazenamentodatalake26.dfs.core.windows.net/SalesLT/"):
    table_name.append(i.name.split('/')[0])


In [0]:
#table_name

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

for i in table_name:
    path = f"abfss://bronze@armazenamentodatalake26.dfs.core.windows.net/SalesLT/{i}/"
    
    df = spark.read.format("parquet").load(path)

    for col in df.columns:
        if "date" in col.lower():
            df = df.withColumn(
                col,
                date_format(
                    from_utc_timestamp(df[col].cast(TimestampType()), "UTC"),
                    "yyyy-MM-dd"
                )
            )

    output_path = f"abfss://silver@armazenamentodatalake26.dfs.core.windows.net/SalesLT/{i}/"

    df.write \
      .format("delta") \
      .mode("overwrite") \
      .save(output_path)

In [0]:
from delta.tables import DeltaTable

DeltaTable.forPath(
  spark,
  "abfss://silver@armazenamentodatalake26.dfs.core.windows.net/SalesLT/SalesOrderHeader/"
)


In [0]:
#dbutils.fs.ls("abfss://silver@armazenamentodatalake26.dfs.core.windows.net/SalesLT/")

'\ndbutils.fs.ls(\n  "abfss://silver@armazenamentodatalake26.dfs.core.windows.net/SalesLT/"\n)\n'